In [140]:
from dotenv import load_dotenv
import os
import json

In [19]:
from pydantic import BaseModel

In [10]:
from openai import OpenAI

In [12]:
client = OpenAI()

In [3]:
load_dotenv()

True

In [6]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [14]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

In [17]:
print(completion.choices[0].message.content)

Function calls itself,  
Layers deep, the logic flows—  
Infinite echo.


In [95]:
completion

ParsedChatCompletion[TaskList](id='chatcmpl-A4bHDeecFYLUd6VZVq0Fsij322cra', choices=[ParsedChoice[TaskList](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[TaskList](content='{"taskt_list":["Write a function to reverse a string.","Implement a basic calculator that can perform addition, subtraction, multiplication, and division.","Create a simple to-do list application with a command-line interface.","Develop a program that sorts a list of numbers using the bubble sort algorithm.","Build a command-line tool that fetches weather data from an API and displays it to the user."],"length":5}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=TaskList(taskt_list=['Write a function to reverse a string.', 'Implement a basic calculator that can perform addition, subtraction, multiplication, and division.', 'Create a simple to-do list application with a command-line interface.', 'Develop a program that sorts a list of numbers using the bu

In [120]:
class TaskList(BaseModel):
    task_list: list[str]
    length: int

In [123]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Give me a list of 5 programming tasks for which a pypi module probably exists."},
    ],
    temperature=0.5
    response_format=TaskList,
)

In [143]:
def generate_task_list(n_items):
    print(f"generating a list of {n_items} programming tasks...")
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Give me a list of {n_items} programming tasks for which PyPI modules probably don't exist."},
        ],
        response_format=TaskList,
    )
    print('completed')
    return completion.choices[0].message.parsed

In [144]:
hard_programming_tasks = generate_task_list(500)


generating a list of 500 programming tasks...
completed


In [145]:
hard_programming_tasks.task_list

['Implement a genetic algorithm for optimizing architectural designs of skyscrapers.',
 'Design a neural network to simulate quantum entanglement in particles.',
 'Create a simulation for simulating the behavior of complex adaptive systems in multi-level environments.',
 'Develop a tool to model the effect of gravitational lensing on observed galaxy images.',
 'Simulate the growth and fractal-like structures of ice crystals under varying atmospheric conditions.',
 'Build a dynamic ecosystem simulator that models predator-prey interactions in real-time.',
 'Create a software that models the spread of misinformation across multiple social media networks.',
 'Develop an algorithm to optimize flight paths in a city for urban air mobility devices.',
 'Implement a symbolic reasoning engine for translating ancient hieroglyphics without annotations.',
 'Design a distributed ledger system specifically for tracking gene editing changes in crops.',
 'Build a system that predicts the evolutionary 

In [124]:
print(completion.choices[0].message.parsed)

task_list=['Download a file from the internet given a URL.', 'Convert Markdown documents to HTML.', 'Plot a graph from a list of data points.', 'Parse JSON data from a string or file.', 'Send an email through an SMTP server.'] length=5


In [126]:
completion.usage

CompletionUsage(completion_tokens=52, prompt_tokens=91, total_tokens=143)

In [127]:
task_list = completion.choices[0].message.parsed

5

In [83]:
batch_input_file = client.files.create(
  file=open("data/batch.jsonl", "rb"),
  purpose="batch"
)

In [84]:
batch_input_file_id = batch_input_file.id

batch = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "test of coding task generation"
    }
)

id = batch.id
print(batch)

Batch(id='batch_E1fP1G9UDldmJk2OJvPSxdhV', completion_window='24h', created_at=1725913400, endpoint='/v1/chat/completions', input_file_id='file-oZ8hcbyx0d3amWaa3I8rKZL3', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1725999800, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'test of coding task generation'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [86]:
print(client.batches.retrieve(id))

Batch(id='batch_E1fP1G9UDldmJk2OJvPSxdhV', completion_window='24h', created_at=1725913400, endpoint='/v1/chat/completions', input_file_id='file-oZ8hcbyx0d3amWaa3I8rKZL3', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1725913412, error_file_id=None, errors=None, expired_at=None, expires_at=1725999800, failed_at=None, finalizing_at=1725913411, in_progress_at=1725913400, metadata={'description': 'test of coding task generation'}, output_file_id='file-ppVheoPS4iAZ9kJT7mqtK075', request_counts=BatchRequestCounts(completed=2, failed=0, total=2))


In [89]:
print(client.batches.retrieve(id).status)


completed


In [90]:
output_file_id = client.batches.retrieve(id).output_file_id

In [116]:
file_response = client.files.content(output_file_id)
print(file_response.text)

{"id": "batch_req_R8AwHaPmy1F3SsIF4a5n7lPa", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "9a9671c347ce18732939598a8b6368ea", "body": {"id": "chatcmpl-A5fCuOPUmfcVFkGQu2MIoF2tgXWLp", "object": "chat.completion", "created": 1725913404, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\"tasks\":[\"Data manipulation and analysis (e.g., using pandas)\",\"Web scraping (e.g., using BeautifulSoup or Scrapy)\",\"Image processing (e.g., using Pillow or OpenCV)\",\"Building REST APIs (e.g., using Flask or FastAPI)\",\"Machine learning (e.g., using scikit-learn or TensorFlow)\"],\"length\":5}", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 80, "completion_tokens": 77, "total_tokens": 157}, "system_fingerprint": "fp_483d39d857"}}, "error": null}
{"id": "batch_req_Iexn9zz10EohbQQ6EEcXl0lQ", "custom_id": "request-2", "response": {"status_code": 200, "request_id": "612bc608

b'{"id": "batch_req_R8AwHaPmy1F3SsIF4a5n7lPa", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "9a9671c347ce18732939598a8b6368ea", "body": {"id": "chatcmpl-A5fCuOPUmfcVFkGQu2MIoF2tgXWLp", "object": "chat.completion", "created": 1725913404, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\\"tasks\\":[\\"Data manipulation and analysis (e.g., using pandas)\\",\\"Web scraping (e.g., using BeautifulSoup or Scrapy)\\",\\"Image processing (e.g., using Pillow or OpenCV)\\",\\"Building REST APIs (e.g., using Flask or FastAPI)\\",\\"Machine learning (e.g., using scikit-learn or TensorFlow)\\"],\\"length\\":5}", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 80, "completion_tokens": 77, "total_tokens": 157}, "system_fingerprint": "fp_483d39d857"}}, "error": null}\n{"id": "batch_req_Iexn9zz10EohbQQ6EEcXl0lQ", "custom_id": "request-2", "response": {"status_code": 200, "reque

In [109]:
file_response.text

'{"id": "batch_req_R8AwHaPmy1F3SsIF4a5n7lPa", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "9a9671c347ce18732939598a8b6368ea", "body": {"id": "chatcmpl-A5fCuOPUmfcVFkGQu2MIoF2tgXWLp", "object": "chat.completion", "created": 1725913404, "model": "gpt-4o-mini-2024-07-18", "choices": [{"index": 0, "message": {"role": "assistant", "content": "{\\"tasks\\":[\\"Data manipulation and analysis (e.g., using pandas)\\",\\"Web scraping (e.g., using BeautifulSoup or Scrapy)\\",\\"Image processing (e.g., using Pillow or OpenCV)\\",\\"Building REST APIs (e.g., using Flask or FastAPI)\\",\\"Machine learning (e.g., using scikit-learn or TensorFlow)\\"],\\"length\\":5}", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 80, "completion_tokens": 77, "total_tokens": 157}, "system_fingerprint": "fp_483d39d857"}}, "error": null}\n{"id": "batch_req_Iexn9zz10EohbQQ6EEcXl0lQ", "custom_id": "request-2", "response": {"status_code": 200, "reques

In [115]:
file_response

In [94]:
file_response.write_to_file('data/batch_output.jsonl')

In [146]:
# write easy_programming_tasks array to a file
with open('data/hard_programming_tasks.json', 'w') as f:
    json.dump(hard_programming_tasks.task_list, f)